In [2]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [3]:
import pandas as pd


df = pd.read_csv('/Users/nick/Downloads/wfp_food_prices_ken.csv')

df.head()



,date,admin1,admin2,market,market_id,latitude,longitude,category,commodity,commodity_id,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#loc+market+code,#geo+lat,#geo+lon,#item+type,#item+name,#item+code,#item+unit,#item+price+flag,#item+price+type,#currency+code,#value,#value+usd
1,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,cereals and tubers,Maize,51,KG,actual,Wholesale,KES,16.13,0.22
2,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,pulses and nuts,Beans,50,KG,actual,Wholesale,KES,33.63,0.47
3,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,pulses and nuts,Beans (dry),262,90 KG,actual,Wholesale,KES,3246,45.15
4,2006-01-15,Eastern,Kitui,Kitui,187,-1.37,38.02,cereals and tubers,Maize (white),67,KG,actual,Retail,KES,17,0.24


In [4]:
df.columns

Index(['date', 'admin1', 'admin2', 'market', 'market_id', 'latitude',
       'longitude', 'category', 'commodity', 'commodity_id', 'unit',
       'priceflag', 'pricetype', 'currency', 'price', 'usdprice'],
      dtype='object')

In [5]:

df.rename(columns={
    'admin1': 'county',
    'admin2' : 'town',
    'usdprice' : 'usd_price'
}, inplace=True)

In [6]:
#Converting date to datetime
df['date']=pd.to_datetime(df['date'], errors='coerce')

/var/folders/46/kjst_0fj3d9gz7hj7my_685r0000gn/T/ipykernel_56529/4013781289.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date']=pd.to_datetime(df['date'], errors='coerce')


In [7]:
#understanding the column data types
df.dtypes

date            datetime64[ns]
county                  object
town                    object
market                  object
market_id               object
latitude                object
longitude               object
category                object
commodity               object
commodity_id            object
unit                    object
priceflag               object
pricetype               object
currency                object
price                   object
usd_price               object
dtype: object

In [8]:
# Changing price to numeric value and dropping rows with missing values

df['price']=pd.to_numeric(df['price'], errors='coerce')

df = df.dropna(subset=['price', 'commodity', 'market', 'date'])

df.head()

,date,county,town,market,market_id,latitude,longitude,category,commodity,commodity_id,unit,priceflag,pricetype,currency,price,usd_price
1,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,cereals and tubers,Maize,51,KG,actual,Wholesale,KES,16.13,0.22
2,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,pulses and nuts,Beans,50,KG,actual,Wholesale,KES,33.63,0.47
3,2006-01-15,Coast,Mombasa,Mombasa,191,-4.05,39.67,pulses and nuts,Beans (dry),262,90 KG,actual,Wholesale,KES,3246.00,45.15
4,2006-01-15,Eastern,Kitui,Kitui,187,-1.37,38.02,cereals and tubers,Maize (white),67,KG,actual,Retail,KES,17.00,0.24
5,2006-01-15,Eastern,Kitui,Kitui,187,-1.37,38.02,cereals and tubers,Potatoes (Irish),148,50 KG,actual,Wholesale,KES,1249.99,17.39


In [12]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

# Load environment variables
load_dotenv()

# Read them into dictionary
DB_CONFIG = {
    "host": os.getenv("PG_HOST"),
    "port": os.getenv("PG_PORT"),
    "user": os.getenv("PG_USER"),
    "password": os.getenv("PG_PASSWORD"),
    "dbname": os.getenv("PG_DATABASE")
}


# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)

In [13]:
def clean_data(df):
    df.columns = df.columns.str.lower().str.strip()
    df.rename(columns={'admin1': 'province', 'admin2': 'county'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

    # Convert numeric columns
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    df['usdprice'] = pd.to_numeric(df['usdprice'], errors='coerce')
    df['commodity_id'] = pd.to_numeric(df['commodity_id'], errors='coerce').astype('Int64')
    df['market_id'] = pd.to_numeric(df['market_id'], errors='coerce').astype('Int64')

    # Drop rows missing key info
    df.dropna(subset=['price', 'usdprice', 'date', 'commodity_id', 'market_id'], inplace=True)
    return df


In [14]:
print(df.columns)

Index(['date', 'county', 'town', 'market', 'latitude', 'longitude', 'category',
       'commodity', 'commodity_id', 'unit', 'priceflag', 'pricetype',
       'currency', 'price', 'usd_price'],
      dtype='object')


In [15]:
def build_star_schema(df):
    
    commodity_dim = df[['commodity_id', 'commodity', 'category']].drop_duplicates()
    market_dim = df[['market_id', 'market', 'province', 'county', 'latitude', 'longitude']].drop_duplicates()

    date_dim = df[['date']].drop_duplicates().copy()
    date_dim['date_key'] = date_dim['date'].dt.strftime('%Y%m%d').astype(int)
    date_dim['year'] = date_dim['date'].dt.year
    date_dim['month'] = date_dim['date'].dt.month
    date_dim['day'] = date_dim['date'].dt.day

    df['date_key'] = df['date'].dt.strftime('%Y%m%d').astype(int)

    fact_food_prices = df[[
        'date_key', 'commodity_id', 'market_id', 'unit', 'price', 'usdprice',
        'pricetype', 'priceflag', 'currency'
    ]]

    return commodity_dim, market_dim, date_dim, fact_food_prices

In [ ]:
## Inflation data

In [16]:
import requests
import pandas as pd


url = "https://api.worldbank.org/v2/country/ke/indicator/FP.CPI.TOTL?format=json&date=2006:2024"

response = requests.get(url)
data = response.json()


records = data[1]


df = pd.DataFrame.from_records([
    {
        "year": int(row['date']),
        "cpi": row['value']
    }
    for row in records if row['value'] is not None
])

df = df.sort_values("year").reset_index(drop=True)

print(df.head())

   year         cpi
0  2006   63.552636
1  2007   69.754661
2  2008   88.058156
3  2009   96.189558
4  2010  100.000000
